In [32]:
import pandas as pd
import numpy as np
import pickle
import gc

In [33]:
def reduce_mem_usage(df,verbose=True):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    start_mem = df.memory_usage().sum() / 1024**2 
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df  


In [34]:
calendar_file = ".\\data\\calendar.csv"
sales_train_validation_file = ".\\data\\sales_train_validation.csv"
sell_prices_file = ".\\data\\sell_prices.csv"

# Load dataset
calendar_ = pd.read_csv(calendar_file, delimiter=",")
sales_train_validation_ = pd.read_csv(sales_train_validation_file, delimiter=",")
sell_prices_ = pd.read_csv(sell_prices_file, delimiter=",")

In [35]:
calendar_ = reduce_mem_usage(calendar_)

Mem. usage decreased to  0.14 Mb (33.2% reduction)


In [36]:
for d in range(1914,1970):
    col = 'd_' + str(d)
    sales_train_validation_[col] = 0
    sales_train_validation_[col] = sales_train_validation_[col].astype(np.int16)

In [37]:
sell_prices_ = reduce_mem_usage(sell_prices_)

Mem. usage decreased to 45.77 Mb (78.1% reduction)


In [38]:
print(calendar_.shape)
calendar_.head()

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [39]:
print(sales_train_validation_.shape)
sales_train_validation_.head()

(30490, 1975)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
print(sell_prices_.shape)
sell_prices_.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


## Prepare Training Data

In [41]:
column_names = list(sales_train_validation_.columns)
train_ = pd.melt(sales_train_validation_, id_vars=column_names[0:6],value_vars=column_names[6:],var_name='d', value_name='sales').dropna()

In [42]:
print(train_.shape)
train_.head()

(60034810, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [43]:
train_ = reduce_mem_usage(train_)
train_.head()

Mem. usage decreased to 1146.75 Mb (72.2% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


#### Merge with calendar

In [44]:
train = pd.merge(train_, calendar_, on='d', how = 'left')

In [45]:
# Remove unused
del(train_)
del(calendar_)
gc.collect()

20

In [46]:
# Show all collumns in dataset
train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')

#### Merge with sell_prices

In [47]:
train = pd.merge(train, sell_prices_, on=["store_id","item_id","wm_yr_wk"], how = 'left')

In [48]:
# Drop NA collumns
train = train.dropna(subset = ['sell_price'])
print(train.shape)

(47735397, 22)


In [49]:
del(sell_prices_)
gc.collect()
# Reduce memory
train = reduce_mem_usage(train)

Mem. usage decreased to 2004.74 Mb (0.0% reduction)


In [50]:
# Remove categorical data
train.d = train['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = train.dtypes.index.tolist()
types = train.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category' and i != 0:
        train[cols[i]] = train[cols[i]].cat.codes

train.drop('date',axis=1,inplace=True)

In [51]:
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,weekday,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
7,HOBBIES_1_008_CA_1_validation,1444,3,1,0,0,1,12,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,0.459961
8,HOBBIES_1_009_CA_1_validation,1445,3,1,0,0,1,2,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,1.559570
9,HOBBIES_1_010_CA_1_validation,1446,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,3.169922
11,HOBBIES_1_012_CA_1_validation,1448,3,1,0,0,1,0,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,5.980469
14,HOBBIES_1_015_CA_1_validation,1451,3,1,0,0,1,4,11101,2,...,1,2011,-1,-1,-1,-1,0,0,0,0.700195


### Generate simple features

In [52]:
def lag_change(df,lag_val):
   
    # sales: lag    
    col = f"lag_{lag_val}"
    df.loc[:,col] = df.groupby(['id'])['sales'].transform(lambda x: x - x.shift(lag_val))
    
    return df

In [53]:
def lag_(df,lag_val):
   
    # sales: lag    
    col = f"lag_{lag_val}"
    df.loc[:,col] = df.groupby(['id'])['sales'].transform(lambda x: x.shift(lag_val))
    
    return df

In [54]:
def rolling_mean_(df,lag_val,days):
   
    # sales: rolling_mean   
    col = f"mov_avg_{days}_{lag_val}"
    df.loc[:,col] = df.loc[:,f"lag_{lag_val}"].transform(lambda x: x.rolling(days).mean())
    
    return df

In [55]:
def simple_features_monthly(df):
    
    # monthly mean, std minimum/max sales
    months = set(df['month'])
    
    for month in months:        
        df.loc[df['month'] == month,'montly_sales_mean'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.mean())
        df.loc[df['month'] == month,'montly_sales_std'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.std())
        df.loc[df['month'] == month,'monthly_sales_max'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.max())
        df.loc[df['month'] == month,'monthly_sales_min'] = df.loc[df['month'] == month].groupby('id')['sales'].transform(lambda x: x.min())
        print("done")
    
    return df

In [56]:
def simple_features_weekly(df):
    
    # weekly mean, std minimum/max sales, max-min 
    weekdays = set(df['weekday'])
    
    for day in weekdays:
        
        df.loc[df['weekday'] == day,'weekly_sales_mean'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.mean())
        df.loc[df['weekday'] == day,'weekly_sales_std'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.std())
        df.loc[df['weekday'] == day,'weekly_sales_max'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.max())
        df.loc[df['weekday'] == day,'weekly_sales_min'] = df.loc[df['weekday'] == day].groupby('id')['sales'].transform(lambda x: x.min())
        print("done")
        
    return df

In [57]:
def generate_features(df):
    
    start = time.time()
    
    df['item_avg_sales'] = df.groupby('item_id')['sales'].transform('mean').astype(np.float16)
    print("done")    
    df['state_avg_sales'] = df.groupby('state_id')['sales'].transform('mean').astype(np.float16)
    print("done")
    df['store_avg_sales'] = df.groupby('store_id')['sales'].transform('mean').astype(np.float16)
    print("done")
    df['cat_avg_sales'] = df.groupby('cat_id')['sales'].transform('mean').astype(np.float16)
    print("done")
    df['dept_avg_sales'] = df.groupby('dept_id')['sales'].transform('mean').astype(np.float16)
    
    print("done")
    
    df['cat_dept_avg_sales'] = df.groupby(['cat_id','dept_id'])['sales'].transform('mean').astype(np.float16)
    print("done")
    df['cat_item_avg_sales'] = df.groupby(['cat_id','item_id'])['sales'].transform('mean').astype(np.float16)
    print("done")
    print("done")
    df['dept_item_avg_sales'] = df.groupby(['dept_id','item_id'])['sales'].transform('mean').astype(np.float16)
    print("done")
    df['state_store_avg_sales'] = df.groupby(['state_id','store_id'])['sales'].transform('mean').astype(np.float16)
    print("done")
    
    df['state_store_cat_avg_sales'] = df.groupby(['state_id','store_id','cat_id'])['sales'].transform('mean').astype(np.float16)
    print("done")
    df['store_cat_dept_avg_sales'] = df.groupby(['store_id','cat_id','dept_id'])['sales'].transform('mean').astype(np.float16)
    print("done")
    
    end = time.time()
    print(end - start)
    
    return df

In [64]:
def lag_sp(df,lag_val):
   
    # sales: lag    
    col_ = f"lag_{lag_val}_price"
    df.loc[:,col_] = df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(lag_val))
    
    return df

def lag_change_sp(df,lag_val):
   
    # sales: lag    
    col = f"lag_{lag_val}"
    df.loc[:,col] = df.groupby(['id'])['sell_price'].transform(lambda x: x - x.shift(lag_val))
    
    return df

def rolling_mean_sp(df,lag_val,days):
   
    # sales: rolling_mean   
    col_ = f"mov_avg_{days}_{lag_val}_price"
    df.loc[:,col_] = df.loc[:,f"lag_{lag_val}_price"].transform(lambda x: x.rolling(days).mean())  

    return df

def simple_features_monthly_sp(df):
    
    # monthly mean, std minimum/max sales
    months = set(df['month'])
    
    for month in months:        
        
        df.loc[df['month'] == month,'montly_price_mean'] = df.loc[df['month'] == month].groupby('id')['sell_price'].transform(lambda x: x.mean())
        df.loc[df['month'] == month,'montly_price_std'] = df.loc[df['month'] == month].groupby('id')['sell_price'].transform(lambda x: x.std())
        df.loc[df['month'] == month,'monthly_price_max'] = df.loc[df['month'] == month].groupby('id')['sell_price'].transform(lambda x: x.max())
        df.loc[df['month'] == month,'monthly_price_min'] = df.loc[df['month'] == month].groupby('id')['sell_price'].transform(lambda x: x.min())
      
        print("done")
    
    return df

def simple_features_weekly_sp(df):
    
    # weekly mean, std minimum/max sales, max-min 
    weekdays = set(df['weekday'])
    
    for day in weekdays:
        
        df.loc[df['weekday'] == day,'weekly_price_mean'] = df.loc[df['weekday'] == day].groupby('id')['sell_price'].transform(lambda x: x.mean())
        df.loc[df['weekday'] == day,'weekly_price_std'] = df.loc[df['weekday'] == day].groupby('id')['sell_price'].transform(lambda x: x.std())
        df.loc[df['weekday'] == day,'weekly_price_max'] = df.loc[df['weekday'] == day].groupby('id')['sell_price'].transform(lambda x: x.max())
        df.loc[df['weekday'] == day,'weekly_price_min'] = df.loc[df['weekday'] == day].groupby('id')['sell_price'].transform(lambda x: x.min())
                
        print("done")
        
    return df

In [59]:
# Save as pickle
train.to_pickle("train.pkl")

In [60]:
train = lag_(train,7)
train = lag_(train,28)

train = lag_change(train,7)
train = lag_change(train,14)
train = lag_change(train,21)
train = lag_change(train,28)

train = rolling_mean_(train,7,7)
train = rolling_mean_(train,7,28)
train = rolling_mean_(train,28,7)
train = rolling_mean_(train,28,28)

In [61]:
#Reduce memory
train = reduce_mem_usage(train)

Mem. usage decreased to 2368.73 Mb (48.0% reduction)


In [62]:
# Monthly and weekly
train = simple_features_monthly(train)
train = simple_features_weekly(train)
train = reduce_mem_usage(train)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
Mem. usage decreased to 3097.12 Mb (41.4% reduction)


In [65]:
# Sell price features
train = lag_sp(train,7)
train = lag_sp(train,28)

train = lag_change_sp(train,7)
train = lag_change_sp(train,14)
train = lag_change_sp(train,21)
train = lag_change_sp(train,28)

train = rolling_mean_sp(train,7,7)
train = rolling_mean_sp(train,7,28)
train = rolling_mean_sp(train,28,7)
train = rolling_mean_sp(train,28,28)
train = reduce_mem_usage(train)

Mem. usage decreased to 3643.40 Mb (23.1% reduction)


In [66]:
train = simple_features_monthly_sp(train)
train = simple_features_weekly_sp(train)
train = reduce_mem_usage(train)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
Mem. usage decreased to 4371.79 Mb (33.3% reduction)


In [67]:
train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'sell_price', 'lag_7', 'lag_28', 'lag_14', 'lag_21',
       'mov_avg_7_7', 'mov_avg_28_7', 'mov_avg_7_28', 'mov_avg_28_28',
       'montly_sales_mean', 'montly_sales_std', 'monthly_sales_max',
       'monthly_sales_min', 'weekly_sales_mean', 'weekly_sales_std',
       'weekly_sales_max', 'weekly_sales_min', 'lag_7_price', 'lag_28_price',
       'mov_avg_7_7_price', 'mov_avg_28_7_price', 'mov_avg_7_28_price',
       'mov_avg_28_28_price', 'montly_price_mean', 'montly_price_std',
       'monthly_price_max', 'monthly_price_min', 'weekly_price_mean',
       'weekly_price_std', 'weekly_price_max', 'weekly_price_min'],
      dtype='object')

In [68]:
# Save as pickle
train.to_pickle("train_complete.pkl")

In [69]:
print(np.unique(train['event_name_1'].values))

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29]


In [56]:
train_data.to_pickle("D://LSDA_pickles//lag_rolling_mean_features.pkl")

In [70]:
train_data = simple_features_monthly(train_data)

done
done
done
done
done
done
done
done
done
done
done
done


In [71]:
train_data.to_pickle("D://LSDA_pickles//lag_rolling_mean_monthly_features.pkl")

In [72]:
train_data = simple_features_weekly(train_data)

done
done
done
done
done
done
done


In [73]:
train_data.to_pickle("D://LSDA_pickles//lag_rolling_mean_monthly_weekly_features.pkl")

In [74]:
train_data.shape

(46027957, 29)

In [77]:
train_data.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'sell_price', 'lag_7', 'lag_28', 'mov_avg_7_7', 'mov_avg_28_7',
       'mov_avg_7_28', 'mov_avg_28_28', 'montly_sales_mean',
       'montly_sales_std', 'monthly_sales_max', 'monthly_sales_min',
       'weekly_sales_mean', 'weekly_sales_std', 'weekly_sales_max',
       'weekly_sales_min'],
      dtype='object')

In [79]:
58327370-46027957

12299413

In [82]:
train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,mov_avg_7_28,mov_avg_28_28,montly_sales_mean,montly_sales_std,monthly_sales_max,monthly_sales_min,weekly_sales_mean,weekly_sales_std,weekly_sales_max,weekly_sales_min
0,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,NaN,NaN,8.500000,10.676929,75.0,0.0,8.613139,8.786226,47.0,0.0
1,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_2,15,2011-01-30,11101,...,NaN,NaN,8.500000,10.676929,75.0,0.0,6.394161,7.405431,45.0,0.0
2,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,...,NaN,NaN,8.500000,10.676929,75.0,0.0,7.058608,9.588033,54.0,0.0
3,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,...,NaN,NaN,9.170588,11.581472,91.0,0.0,6.542125,8.202350,50.0,0.0
4,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,...,NaN,NaN,9.170588,11.581472,91.0,0.0,7.560440,9.495985,48.0,0.0


In [83]:
grouped_ = train_data.groupby('id')

In [84]:
item_df = (grouped_.get_group('FOODS_3_586_CA_1_validation'))

In [93]:
item_df[100:140]

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,mov_avg_7_28,mov_avg_28_28,montly_sales_mean,montly_sales_std,monthly_sales_max,monthly_sales_min,weekly_sales_mean,weekly_sales_std,weekly_sales_max,weekly_sales_min
1328049,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_101,36,2011-05-09,11115,...,23.571429,6.964286,45.567742,14.898365,87.0,17.0,40.673993,11.415931,76.0,20.0
1328050,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_102,19,2011-05-10,11115,...,30.142857,8.607143,45.567742,14.898365,87.0,17.0,36.153846,10.068066,66.0,0.0
1328051,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_103,28,2011-05-11,11115,...,32.857143,9.321429,45.567742,14.898365,87.0,17.0,36.692308,10.628097,72.0,0.0
1328052,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_104,34,2011-05-12,11115,...,36.571429,10.214286,45.567742,14.898365,87.0,17.0,38.838828,11.377369,85.0,0.0
1328053,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_105,62,2011-05-13,11115,...,41.857143,11.464286,45.567742,14.898365,87.0,17.0,45.677656,14.006516,133.0,0.0
1431857,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_106,51,2011-05-14,11116,...,9.142857,2.535714,45.567742,14.898365,87.0,17.0,58.076642,13.507750,111.0,30.0
1431858,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_107,68,2011-05-15,11116,...,18.857143,5.000000,45.567742,14.898365,87.0,17.0,64.649635,16.581949,114.0,0.0
1431859,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_108,33,2011-05-16,11116,...,24.714286,6.500000,45.567742,14.898365,87.0,17.0,40.673993,11.415931,76.0,20.0
1431860,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_109,29,2011-05-17,11116,...,28.714286,7.464286,45.567742,14.898365,87.0,17.0,36.153846,10.068066,66.0,0.0
1431861,FOODS_3_586_CA_1_validation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_110,26,2011-05-18,11116,...,33.857143,8.750000,45.567742,14.898365,87.0,17.0,36.692308,10.628097,72.0,0.0
